In [1]:
import numpy as np
import scipy as sp
import sys
from mendeleev import element
import mdtraj as md

In [2]:
%load_ext autoreload
sys.path.append("/home/gmancini/Dropbox/appunti/old_search_algos_28_08_2020/molecule_utils")
import zmatrix, quaternion_utils
%autoreload 2

In [3]:
har2kjmol = sp.constants.physical_constants["Avogadro constant"][0]\
*sp.constants.physical_constants["Hartree energy"][0]/1000.
har2kjmol

2625.4996394798254

In [4]:
covalent_radii = dict(zip(("C","N","O"),\
                     [0.01*(lambda x: element(x).covalent_radius_bragg)(i) for i in ("C","N","O")]))
covalent_radii['H'] = 0.01*element('H').covalent_radius_pyykko
covalent_radii

{'C': 0.77, 'N': 0.65, 'O': 0.65, 'H': 0.32}

In [5]:
zeff = dict(zip(("C","H","N","O"),[(lambda x: element(x).zeff(o='s')-0.4)(i) for i in ("C","H","N","O")]))
zeff

{'C': 2.85, 'H': 0.6, 'N': 3.5000000000000004, 'O': 4.1499999999999995}

In [6]:
#elec = dict(zip(("C","H","N","O"),\
#                     [(lambda x: element(x).electronegativity(scale='allred-rochow'))\
#                     (i) for i in ("C","H","N","O")]))
# allrd-rochov scale is not in the expected values or units
# quick & dirty estimate from eq. and adjust zeff as above
elec =dict(zip(("C","H","N","O"),(2.5,2.2,3.07,3.50)))
elec

{'C': 2.5, 'H': 2.2, 'N': 3.07, 'O': 3.5}

In [7]:
tol = 0.4

In [8]:
tempo = md.load("tempo.pdb")
tempo_top = tempo.topology
tempo_atoms = [a.element.symbol for a in tempo_top.atoms]
tempo.xyz

array([[[ 0.0694,  0.0206,  0.    ],
        [ 0.0022,  0.0025,  0.1318],
        [-0.1434,  0.05  ,  0.1238],
        [-0.2166,  0.0006,  0.    ],
        [-0.1434,  0.05  , -0.1238],
        [ 0.0022,  0.0025, -0.1318],
        [ 0.0108, -0.1451,  0.1733],
        [ 0.078 ,  0.0877,  0.2336],
        [ 0.078 ,  0.0877, -0.2336],
        [ 0.0108, -0.1451, -0.1733],
        [ 0.1961,  0.0067,  0.    ],
        [-0.2238, -0.1089,  0.    ],
        [-0.3196,  0.038 ,  0.    ],
        [-0.1944,  0.0182,  0.2155],
        [-0.1445,  0.1599,  0.1237],
        [-0.1944,  0.0182, -0.2155],
        [-0.1445,  0.1599, -0.1237],
        [ 0.0798,  0.1926, -0.2025],
        [ 0.1811,  0.0533, -0.2441],
        [ 0.028 ,  0.081 , -0.3308],
        [ 0.1145, -0.1793, -0.1662],
        [-0.0514, -0.2098, -0.1109],
        [-0.0224, -0.1563, -0.2771],
        [-0.0514, -0.2098,  0.1109],
        [ 0.1145, -0.1793,  0.1662],
        [-0.0224, -0.1563,  0.2771],
        [ 0.1811,  0.0533,  0.2441],
 

In [9]:
tempo_zm = zmatrix.zmat("tempo.zmat",((2,7),(2,8),(6,9),(6,10)),fmt="zmat")
tempo_zm.admat.shape

(31, 31)

In [10]:
ch3oh = md.load("ch3oh.pdb")
ch3oh_top = ch3oh.topology
ch3oh_atoms = [a.element.symbol for a in ch3oh_top.atoms]

In [11]:
ch3oh_top.select("resid 1")

array([ 6,  7,  8,  9, 10, 11])

In [12]:
def create_hole(solute,solute_top,solvent,solvent_top,radii,elec,tol):
    """
    create hole using covalent radii and same eq as proxima with
    role of tolerance reversed
    """
    solute_atoms = [a.element.symbol for a in solute_top.atoms]
    solvent_atoms = [a.element.symbol for a in solvent_top.atoms]
    solvent_natoms = len(solvent_top.select("resid 1"))
    residues = list(range(solvent_top.n_residues))
    N = len(solvent_top.select("resid 1"))
    remove = list()
    for res in range(len(residues)):
        ratoms = solvent_top.select("resid " + str(res))
        for jatom in range(solute.n_atoms):
            jelem = solute_atoms[jatom]
            if jelem == "VS": continue
            D = 10.*np.linalg.norm(solvent.xyz[0][ratoms]-solute.xyz[0][jatom],axis=1)
            C = np.array([(radii[i] + radii[jelem]-0.07*(elec[i]-elec[jelem])**2 + tol)\
                           for i in solvent_atoms[:N]])
            if np.min(D) < np.max(C):
                remove.append(res)
    okres = list(set(residues).difference(remove))
    rlist = "resid " + ' '.join(list(map(str,okres)))
    solvent.restrict_atoms(solvent_top.select(rlist))
    return okres

In [13]:
hole = create_hole(tempo,tempo_top,ch3oh,ch3oh_top,covalent_radii,elec,tol)
len(hole)

494

In [14]:
ch3oh.save("test.xyz")

In [15]:
tempo.save("1.xyz")